# Preprocesado de Datos
 ## Se importan datos de BBDD usando las funcionalidades de Numpy y Pandas
 
 ### Trabajo de Fin de Grado - Universidad de Burgos
 #### Jesús García Armario

In [1]:
## Imports
import pandas as pd
import numpy as np

### Dataset Servicio Andaluz de Salud:

In [2]:
# Cargamos un dataframe por especialidad
ruta ='.\Listados'
df_plastica = pd.read_excel(ruta+'\listadoPlastica.xls', skipfooter= 12, header = [9,10], index_col=[1,4,5])
df_neurocia = pd.read_excel(ruta+'\listadoNeurocia.xls', skipfooter= 12, header = [9,10], index_col=[1,4,5])
df_toracica = pd.read_excel(ruta+'\listadoToracica.xls', skipfooter= 12, header = [9,10], index_col=[1,4,5])
df_maxilo = pd.read_excel(ruta+'\listadoMaxilo.xls', skipfooter= 12, header = [9,10], index_col=[1,4,5])
df_trauma = pd.read_excel(ruta+'\listadoTrauma.xls', skipfooter= 12, header = [9,10], index_col=[1,4,5])
df_otorrino = pd.read_excel(ruta+'\listadoOtorrino.xls', skipfooter= 12, header = [9,10], index_col=[1,4,5])
df_general = pd.read_excel(ruta+'\listadoGeneral.xls', skipfooter= 12, header = [9,10], index_col=[1,4,5])

In [3]:
# Definimos función que recibe el data frame y lo devuelve preprocesado
def preprocesar(df: pd.DataFrame):
    # Eliminamos columnas vacías
    df = df.droplevel(level=1,axis=0)
    df = df.droplevel(level = 1, axis = 1)
    df = df.dropna(axis=1, how='all')
    del df['QUIRÓFANO']
    del df['APELLIDOS, NOMBRE']
    del df['TURNO']
    del df['INICIO']
    del df['FIN']
    df = df.drop(df.columns[[0]],axis=1)

    # Se renombran las columnas repetidas
    cols = []
    count = 1
    countT = 1
    contC = 1
    for i in df.columns:
        if i=='DURACIÓN':
            cols.append('DURACIÓN'+str(count))
            count += 1
            continue
        if i == 'TIPO':
            cols.append('TIPO'+str(countT))
            countT+=1
            continue
        if i=='CARÁCTER ECONÓMICO':
            cols.append('CARÁCTER ECONÓMICO'+str(contC))
            contC+=1
            continue
        cols.append(i)

    df.columns = cols

    # Eliminamos la columna que sobra
    del df['DURACIÓN1']
    # Renombramos la otra
    df.rename(columns = {"TIPO1": "TIPO", "TIPO2": "TURNO", "DURACIÓN2":"DURACIÓN", "CARÁCTER ECONÓMICO1":"CARÁCTER ECONÓMICO",
    "CARÁCTER ECONÓMICO2":"PONDERACIÓN"},inplace=True)

    # Eliminamos filas vacías
    df = df.dropna(axis=0,how = 'all')
    # Codificamos: Cirugía Menor = 0; Cirugía Mayor = 1
    df['TIPO'] = df['TIPO'].replace(['Menor','Mayor'],[0,1])
    # Codificamos: Mañana = 0; Tarde = 1
    df['TURNO'] = df['TURNO'].replace(['Mañana','Tarde'],[0,1])
    # Codificamos: Actividad Ordinaria = 0; Continuidad Asistencial = 1
    df['CARÁCTER ECONÓMICO'] = df['CARÁCTER ECONÓMICO'].replace(['Actividad Ordinaria','Continuidad Asistencial'],[0,1])
    # Eliminamos texto
    df['TIPO'] = df['TIPO'].replace(r'^\D',np.nan,regex=True)
    df['CARÁCTER ECONÓMICO'] = df['CARÁCTER ECONÓMICO'].replace(r'^\D',np.nan,regex=True)
    df['PONDERACIÓN'] = df['PONDERACIÓN'].replace(r'^\D',np.nan,regex=True)
    df['NUHSA'] = df['NUHSA'].replace(r'^\D',np.nan,regex=True)
    # Eliminamos filas nulas
    df = df.dropna(axis=0,how = 'all')

    # Cambiamos el nombre del atributo NUHSA
    df.rename(columns={"NUHSA":"INTERVENCIÓN"},inplace=True)
    df.reset_index(inplace = True)
    del df["level_0"]
    df.rename(columns={"level_1":"NHC"},inplace = True)
    df = df.groupby('NHC', as_index = False).aggregate({"INTERVENCIÓN":"last","TIPO":"first", "TURNO":"first", 
    "CARÁCTER ECONÓMICO":"last","PONDERACIÓN":"last", "DURACIÓN":"first"})

    # Eliminamos los valores que tienen algún nulo
    df = df.dropna(axis=0,how='any')

    return df

In [4]:
# Preprocesamos los datos
df_plastica = preprocesar(df_plastica)
df_neurocia = preprocesar(df_neurocia)
df_toracica = preprocesar(df_toracica)
df_maxilo = preprocesar(df_maxilo)
df_trauma = preprocesar(df_trauma)
df_otorrino = preprocesar(df_otorrino)
df_general = preprocesar(df_general)
# Añadimos una columna con el valor de las especialidades
especialidad = ['PLASTICA'] * len(df_plastica)
df_plastica['ESPECIALIDAD'] = especialidad
especialidad = ['NEUROCIRUGIA'] * len(df_neurocia)
df_neurocia['ESPECIALIDAD'] = especialidad
especialidad = ['TORACICA'] * len(df_toracica)
df_toracica['ESPECIALIDAD'] = especialidad
especialidad = ['MAXILOFACIAL'] * len(df_maxilo)
df_maxilo['ESPECIALIDAD'] = especialidad
especialidad = ['TRAUMATOLOGIA'] * len(df_trauma)
df_trauma['ESPECIALIDAD'] = especialidad
especialidad = ['OTORRINOLARINGOLOGIA'] * len(df_otorrino)
df_otorrino['ESPECIALIDAD'] = especialidad
especialidad = ['GENERAL'] * len(df_general)
df_general['ESPECIALIDAD'] = especialidad

In [5]:
# Unimos los dataframes en uno solo
df = pd.concat([df_plastica,df_neurocia,df_toracica,df_maxilo,df_trauma, df_otorrino, df_general],ignore_index=True)

EXPORTAMOS A ARCHIVO EXTERNO

In [6]:
# Exportamos a Excel
archivo = "ListadoInterv_Preprocesado.xlsx"
df.to_excel(archivo)

DIVISIÓN POR ESPECIALIDADES

In [7]:
# Cargamos el archivo preprocesado
archivo = "ListadoInterv_Preprocesado.xlsx"
df = pd.read_excel(archivo)

In [8]:
# Creamos un dataframe por cada especialidad
df_plastica = df[df['ESPECIALIDAD'] == 'PLASTICA']
df_neurocia = df[df['ESPECIALIDAD'] == 'NEUROCIRUGIA']
df_toracica = df[df['ESPECIALIDAD'] == 'TORACICA']
df_maxilo = df[df['ESPECIALIDAD'] == 'MAXILOFACIAL']
df_trauma = df[df['ESPECIALIDAD'] == 'TRAUMATOLOGIA']
df_otorrino = df[df['ESPECIALIDAD'] == 'OTORRINOLARINGOLOGIA']
df_general = df[df['ESPECIALIDAD'] == 'GENERAL']

In [9]:
# Exportamos cada dataframe a un archivo Excel
archivo = "ListadoInterv_Preprocesado_PLASTICA.xlsx"
df_plastica.to_excel(archivo)
archivo = "ListadoInterv_Preprocesado_NEUROCIRUGIA.xlsx"
df_neurocia.to_excel(archivo)
archivo = "ListadoInterv_Preprocesado_TORACICA.xlsx"
df_toracica.to_excel(archivo)
archivo = "ListadoInterv_Preprocesado_MAXILOFACIAL.xlsx"
df_maxilo.to_excel(archivo)
archivo = "ListadoInterv_Preprocesado_TRAUMATOLOGIA.xlsx"
df_trauma.to_excel(archivo)
archivo = "ListadoInterv_Preprocesado_OTORRINOLARINGOLOGIA.xlsx"
df_otorrino.to_excel(archivo)
archivo = "ListadoInterv_Preprocesado_GENERAL.xlsx"
df_general.to_excel(archivo)

AMPLIACIÓN DATOS

In [10]:
# Cargamos el datafrma de cirugía plástica
archivo = "ListadoInterv_Preprocesado_PLASTICA.xlsx"
df_plastica = pd.read_excel(archivo)
# Cargamos la actualización de cirugía plástica
ruta = ".\Act_List"
df_plastica_act = pd.read_excel(ruta+"\codigosPlastica.xlsx")

In [11]:
print(df_plastica_act.head())

   Usuario (Nhc) Usuario (Sexo) Usuario (Fecha de nacimiento)     Asa  \
0       153619.0          Mujer                    1966-08-25  ASA II   
1      3628310.0          Mujer                    1964-10-17  ASA II   
2      3085680.0          Mujer                    1961-06-10  ASA II   
3      1252380.0          Mujer                    1960-04-29  ASA II   
4      1884313.0         Hombre                    1984-11-17  ASA II   

  Garantía del procedimiento (Con garantía| Sin garantía)  \
0                                       Sin garantía        
1                                       Sin garantía        
2                                       Sin garantía        
3                                       Sin garantía        
4                                       Sin garantía        

   Código facultativo responsable Código diagnóstico  Código especialidad  \
0                         6224871              V51.0                70240   
1                         6224871       

In [12]:
# Cambiamos el nombre de la columna 'Usuario (NHC)' por 'NHC'
df_plastica_act.rename(columns={"Usuario (Nhc)":"NHC"},inplace=True)
# Eliminamos la columna 'Unnamed: 0 y 'Unnamed: 0.1'
del df_plastica['Unnamed: 0']
del df_plastica['Unnamed: 0.1']
# Eliminamos los valores nulos
df_plastica = df_plastica.dropna(axis=0,how='any')
df_plastica_act = df_plastica_act.dropna(axis=0,how='any')
# Eliminamos los duplicados
df_plastica = df_plastica.drop_duplicates()
df_plastica_act = df_plastica_act.drop_duplicates()

In [13]:
# Hacemos un merge de los dos dataframes
df_plastica = pd.merge(df_plastica,df_plastica_act,how='left',on='NHC')

In [14]:
# Eliminamos los valores nulos
df_plastica = df_plastica.dropna(axis=0,how='any')
print(df_plastica.head())

       NHC  INTERVENCIÓN  TIPO  TURNO  CARÁCTER ECONÓMICO  PONDERACIÓN  \
1  1001107         86.40     1      0                  36           36   
3  1001395         86.40     1      0                  55           55   
4  1001395         86.40     1      0                  55           55   
5  1001623         86.70     1      1                 146          146   
6  1002204         86.74     1      1                  41           41   

   DURACIÓN ESPECIALIDAD Usuario (Sexo) Usuario (Fecha de nacimiento)  \
1        33     PLASTICA         Hombre                    1933-01-31   
3        60     PLASTICA         Hombre                    1936-04-05   
4        60     PLASTICA         Hombre                    1936-04-05   
5        42     PLASTICA          Mujer                    1983-02-16   
6        72     PLASTICA         Hombre                    1935-03-02   

       Asa Garantía del procedimiento (Con garantía| Sin garantía)  \
1  ASA III                                    

In [15]:
# Definimos función para calcular edad a partir de fecha de nacimiento
import datetime
def calcular_edad(fecha_nacimiento):
    hoy = datetime.date.today()
    edad = hoy.year - fecha_nacimiento.year - ((hoy.month, hoy.day) < (fecha_nacimiento.month, fecha_nacimiento.day))
    return edad

In [16]:
# Calculamos la edad a partir de la fecha de nacimiento
df_plastica['EDAD'] = df_plastica['Usuario (Fecha de nacimiento)'].apply(lambda x: calcular_edad(x))
del df_plastica['Usuario (Fecha de nacimiento)']

In [17]:
print(df_plastica.head())

       NHC  INTERVENCIÓN  TIPO  TURNO  CARÁCTER ECONÓMICO  PONDERACIÓN  \
1  1001107         86.40     1      0                  36           36   
3  1001395         86.40     1      0                  55           55   
4  1001395         86.40     1      0                  55           55   
5  1001623         86.70     1      1                 146          146   
6  1002204         86.74     1      1                  41           41   

   DURACIÓN ESPECIALIDAD Usuario (Sexo)      Asa  \
1        33     PLASTICA         Hombre  ASA III   
3        60     PLASTICA         Hombre  ASA III   
4        60     PLASTICA         Hombre  ASA III   
5        42     PLASTICA          Mujer    ASA I   
6        72     PLASTICA         Hombre  ASA III   

  Garantía del procedimiento (Con garantía| Sin garantía)  \
1                                       Con garantía        
3                                       Con garantía        
4                                       Sin garantía       

In [18]:
# Exportamos a Excel
archivo = "ListadoInterv_Preprocesado_PLASTICA.xlsx"
df_plastica.to_excel(archivo)